In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!git clone https://github.com/SKTBrain/KoBERT.git
!pip install -r /content/KoBERT/requirements.txt

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-suiizb0y
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-suiizb0y
Cloning into 'KoBERT'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 231 (delta 33), reused 30 (delta 12), pack-reused 155
Receiving objects: 100% (231/231), 199.27 KiB | 1.92 MiB/s, done.
Resolving deltas: 100% (115/115), done.
     |████████████████████████████████| 46.9 MB 72 kB/s 
     |████████████████████████████████| 344 kB 46.6 MB/s 
     |████████████████████████████████| 1.2 MB 46.3 MB/s 
     |████████████████████████████████| 4.5 MB 47.8 MB/s 
     |████████████████████████████████| 1.3 MB 42.8 MB/s 
     |████████████████████████████████| 2.9 MB 45.3 MB/s 
     |████████████████████████████████| 1.1 MB 49.5 MB/s 
     |████████████████████████████████| 895 kB 65.

In [ ]:
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)
sp('도시적 맥락주의에 입각한 항만공간의 지속가능한 방재안전 및 생태환경 조성과 경제적')

using cached model


['▁도시',
 '적',
 '▁',
 '맥',
 '락',
 '주의',
 '에',
 '▁입',
 '각',
 '한',
 '▁항',
 '만',
 '공간',
 '의',
 '▁지속',
 '가',
 '능',
 '한',
 '▁방',
 '재',
 '안전',
 '▁및',
 '▁생',
 '태',
 '환경',
 '▁조성',
 '과',
 '▁경제적']

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [ x.physical_device_desc for x in local_device_protos if x.device_type == 'GPU']

has_V100 = False
gpus = get_available_gpus()
for gpu in gpus:
    if gpu.find('Tesla V100')!=-1:
        has_V100=True

if not has_V100:
    raise ValueError

ValueError: ignored

In [1]:
!nvidia-smi

Tue Aug 17 23:00:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install sentencepiece==0.1.85
!pip install tokenizers==0.8.1.rc1

     |████████████████████████████████| 1.0 MB 8.0 MB/s 
     |████████████████████████████████| 3.0 MB 7.9 MB/s 


In [3]:
!pip install transformers==3.0.2

     |████████████████████████████████| 769 kB 8.3 MB/s 
     |████████████████████████████████| 895 kB 17.8 MB/s 


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from transformers import *

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
file_path = '/content/drive/MyDrive/dataset/dacon/technic_classifier/'

train=pd.read_csv(file_path+'fixed_okt_train.csv')
test=pd.read_csv(file_path+'fixed_okt_test.csv')
sample_submission=pd.read_csv(file_path+'sample_submission.csv')

In [ ]:
train

,index,label,okt_과제명,okt_내역사업명,okt_요약문_한글키워드,okt_요약문_연구목표,token_요약문_연구목표
0,0,24,"['유전', '정보', '해충', '분류군', '동정']","['농산물', '안전성']","['뉴클레오티드', '염기서열', '분자', '종', '동정', '침샘', '전사체']","['해충', '분류군', '동정', '유입', '확산', '추적']","[('새로운', 'Adjective'), ('해충', 'Noun'), ('분류군',..."
1,1,0,"['대장암', 'trail', '내성', '표적', '인자', '발굴', 'trai...","['지역', '대학', '우수', '과학자', '지원', '사업', '년년']","['대장암', '항암제', '내성', '세포', '사멸', '유전자', '발굴']","['trail', '감수성', '표적', '유전자', '발굴', '내성', '제어'...","[('최종', 'Noun'), ('목표', 'Noun'), ('trail', 'Al..."
2,2,0,"['비목', '셀룰로오스', '식물', '자원', '기능성', '부직포', '고부가...","['혁신', '기업']","['기능성', '룰', '오스', '파이버', '천연섬유', '기능성', '부직포'...","['식물계', '자원', '정련', '최적', '신서', '란', '파이버', '기...","[('식물계', 'Noun'), ('자원', 'Noun'), ('정련', 'Noun..."
3,3,0,"['소화기', '암', '진단', '분자영상', '형광', '프로브']","['창업', '사업화', '연계']","['분자', '진단', '형광', '조영제', '프로브', '항체', '대장암']","['암', '특이적', '바이오', '발굴', '바이오', '프로브', '소화기',...","[('암', 'Modifier'), ('특이', 'Noun'), ('적', 'Suf..."
4,4,0,"['위암', '환자', '항암제', '반응', '예측', 'brca', '발현', ...","['기본', '연구지원']","['brca', '제자리', '부', '합법', '조직', '미세', '배열', '...","['수술', '위암', '환자', '생존', '기간', '개월', '안팎', '항'...","[('수술', 'Noun'), ('이', 'Josa'), ('불가능한', 'Adje..."
...,...,...,...,...,...,...,...
174299,174299,0,"['혈관', '내막', '증식', '치료', '표적', '분자', '발굴']",['중견'],"['혈관', '내막', '증식', '혈관', '개조', '혈관', '평활', '근세...","['증식성', '혈관', '질환', '염증', '반응', '관련성', '혈관', '...","[('증식', 'Noun'), ('성', 'Suffix'), ('혈관', 'Noun..."
174300,174300,0,"['곤란', '물성', '조절', '식품', '유동층', '과립', '복합', '소...",['중견'],"['곤란', '유변', '물성', '유동층', '과립', '물리적', '특성', '...","['곤란', '고령자', '식품', '섭취', '유변', '물성', '물성', '조...","[('삼킴', 'Verb'), ('곤란', 'Noun'), ('을', 'Josa')..."
174301,174301,0,"['식물', '유전체', '정보', '고도화', '핵심', '유전자', '군', '...","['공분야', '학문', '후속', '세대', '양성', '대통령', 'postdo...","['유전자', '정보', '고도화', '대상', '유전자', '군', '분석', '...","['지난', '동안', '여종', '식물', '유전체', '정보', '완성', '공...","[('지난', 'Noun'), ('년', 'Noun'), ('여', 'Josa'),..."
174302,174302,23,"['콩', '유전자원', '내건성', '내습성', '대량', '평가', '체계', ...","['유용', '형질', '대량', '특성', '평가', '육종', '모본', '발굴']","['콩', '가공', '적성', '병', '저항성', '내재해성', '콩', '모자...","['내건', '내습성', '유전자원', '수집', '내건성', '자원', '평가',...","[('내', 'Determiner'), ('건', 'Noun'), ('및', 'No..."


In [7]:
def trans_to_list(text):
  text = re.sub("[\[\]\' ]", "", text)
  text =  text.split(',')
  if text == ['']:
    text = []
  return text

def cut_sent(text, number):
  return text[:number]

In [8]:
train['fixed_okt_과제명'] = train['fixed_okt_과제명'].apply(trans_to_list)
train['fixed_okt_내역사업명'] = train['fixed_okt_내역사업명'].apply(trans_to_list)
train['fixed_okt_요약문_한글키워드'] = train['fixed_okt_요약문_한글키워드'].apply(trans_to_list)
train['fixed_okt_요약문_연구목표'] = train['fixed_okt_요약문_연구목표'].apply(trans_to_list)

test['fixed_okt_과제명'] = test['fixed_okt_과제명'].apply(trans_to_list)
test['fixed_okt_내역사업명'] = test['fixed_okt_내역사업명'].apply(trans_to_list)
test['fixed_okt_요약문_한글키워드'] = test['fixed_okt_요약문_한글키워드'].apply(trans_to_list)
test['fixed_okt_요약문_연구목표'] = test['fixed_okt_요약문_연구목표'].apply(trans_to_list)

In [9]:
train['token_data']=train['fixed_okt_내역사업명']+train['fixed_okt_과제명']+train['fixed_okt_요약문_한글키워드']
test['token_data']=test['fixed_okt_내역사업명']+test['fixed_okt_과제명']+test['fixed_okt_요약문_한글키워드']

In [10]:
train_length = train['token_data'].apply(len)
train_length.head()

print('인풋 길이 최대 값: {}'.format(np.max(train_length)))
print('인풋 길이 최소 값: {}'.format(np.min(train_length)))
print('인풋 길이 평균 값: {:.2f}'.format(np.mean(train_length)))
print('인풋 길이 표준편차: {:.2f}'.format(np.std(train_length)))
print('인풋 길이 중간 값: {}'.format(np.median(train_length)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))


print('인풋 길이 98%: {}'.format(np.percentile(train_length, 98)))

인풋 길이 최대 값: 77
인풋 길이 최소 값: 3
인풋 길이 평균 값: 21.39
인풋 길이 표준편차: 5.94
인풋 길이 중간 값: 21.0
인풋 길이 제 1 사분위: 17.0
인풋 길이 제 3 사분위: 25.0
인풋 길이 98%: 35.0


In [11]:
train_length = test['token_data'].apply(len)
train_length.head()

print('인풋 길이 최대 값: {}'.format(np.max(train_length)))
print('인풋 길이 최소 값: {}'.format(np.min(train_length)))
print('인풋 길이 평균 값: {:.2f}'.format(np.mean(train_length)))
print('인풋 길이 표준편차: {:.2f}'.format(np.std(train_length)))
print('인풋 길이 중간 값: {}'.format(np.median(train_length)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))


print('인풋 길이 98%: {}'.format(np.percentile(train_length, 98)))

인풋 길이 최대 값: 70
인풋 길이 최소 값: 3
인풋 길이 평균 값: 21.40
인풋 길이 표준편차: 5.96
인풋 길이 중간 값: 21.0
인풋 길이 제 1 사분위: 17.0
인풋 길이 제 3 사분위: 25.0
인풋 길이 98%: 35.0


In [12]:
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',  cache_dir='bert_ckpt', do_lower_case=False)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [13]:
def input_str(text):
  return ' '.join(text)

In [14]:
train['data'] = train['token_data'].apply(input_str)
test['data'] = test['token_data'].apply(input_str)

In [15]:
train_length = train['data'].apply(len)
train_length.head()

print('인풋 길이 최대 값: {}'.format(np.max(train_length)))
print('인풋 길이 최소 값: {}'.format(np.min(train_length)))
print('인풋 길이 평균 값: {:.2f}'.format(np.mean(train_length)))
print('인풋 길이 표준편차: {:.2f}'.format(np.std(train_length)))
print('인풋 길이 중간 값: {}'.format(np.median(train_length)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))


print('인풋 길이 98%: {}'.format(np.percentile(train_length, 98)))

인풋 길이 최대 값: 331
인풋 길이 최소 값: 11
인풋 길이 평균 값: 73.92
인풋 길이 표준편차: 21.49
인풋 길이 중간 값: 72.0
인풋 길이 제 1 사분위: 59.0
인풋 길이 제 3 사분위: 87.0
인풋 길이 98%: 122.0


In [16]:
train_length = test['data'].apply(len)
train_length.head()

print('인풋 길이 최대 값: {}'.format(np.max(train_length)))
print('인풋 길이 최소 값: {}'.format(np.min(train_length)))
print('인풋 길이 평균 값: {:.2f}'.format(np.mean(train_length)))
print('인풋 길이 표준편차: {:.2f}'.format(np.std(train_length)))
print('인풋 길이 중간 값: {}'.format(np.median(train_length)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))


print('인풋 길이 98%: {}'.format(np.percentile(train_length, 98)))

인풋 길이 최대 값: 331
인풋 길이 최소 값: 11
인풋 길이 평균 값: 74.01
인풋 길이 표준편차: 21.62
인풋 길이 중간 값: 72.0
인풋 길이 제 1 사분위: 59.0
인풋 길이 제 3 사분위: 88.0
인풋 길이 98%: 123.0


In [17]:
# 0이 아닌 데이터 2번 복사하여 추가, 추가로 오답률이 높은 label 2번 복사하여 추가 
temp = train.copy()
temp1 = temp[temp['label'] != 0]
temp2 = temp[(temp['label'] == 11)|(temp['label'] == 12)|(temp['label'] == 13)|(temp['label'] == 17)|(temp['label'] == 20)|(temp['label'] == 23)|(temp['label'] == 25)|(temp['label'] == 26)|(temp['label'] == 37)|(temp['label'] == 38)]

temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)

In [18]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 64
NUM_EPOCHS = 3
VALID_SPLIT = 0.2
MAX_LEN= 122

In [19]:
from transformers import *
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',  cache_dir='bert_ckpt', do_lower_case=False)

def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict=tokenizer.encode_plus(
    text = sent, 
    add_special_tokens=True, 
    max_length=MAX_LEN, 
    pad_to_max_length=True, 
    return_attention_mask=True,
    truncation = True)
    
    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

# def clean_text(sent):
#     sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
#     return sent_clean

for train_sent, train_label in zip(temp['data'], temp['label']):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAX_LEN=MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
        train_data_labels.append(train_label)
        
    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_input_ids=np.array(input_ids, dtype=int)
train_attention_masks=np.array(attention_masks, dtype=int)
train_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
train_inputs=(train_input_ids, train_attention_masks, train_token_type_ids)
train_labels=np.asarray(train_data_labels, dtype=np.int32)

In [ ]:
# def trans_label(number):
#     vector =  np.array([0] * 46)
#     vector[number] = 1
#     return vector

# train['label_vector'] = train['label'].apply(trans_label)
# train_labels=tf.convert_to_tensor(list(train['label_vector']), dtype=tf.float32)

In [20]:
num = 3
print(train_input_ids[num])
print(train_attention_masks[num])
print(train_token_type_ids[num])
print(tokenizer.decode(train_input_ids[num]))

[   101   9736  26784   9405  26784  18227   9568  21611   9448  18227
  12310   9526   9708  24989   9367  13764  30858  14871   9983 118649
 102574  52015   9367  13764   9708  24989   9983 118649   9678  30858
  17730 102574  52015   9959  29683   9069  13890 119115    102      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [21]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=46)

# 학습 준비하기
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_name = "tf2_bert_classifier"

# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=5)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)\

checkpoint_path = os.path.join(model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


tf2_bert_classifier -- Folder create complete 



In [23]:
cls_model.load_weights('/content/drive/MyDrive/dataset/dacon/technic_classifier/weights.h5')

In [24]:
# 학습과 eval 시작
history = cls_model.fit(train_inputs, train_labels, epochs=3, batch_size=64,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/3
3116/3116 [==============================] - 2901s 931ms/step - loss: 0.0658 - accuracy: 0.9790 - val_loss: 0.0835 - val_accuracy: 0.9776

Epoch 00001: val_accuracy improved from -inf to 0.97761, saving model to tf2_bert_classifier/weights.h5
Epoch 2/3
3116/3116 [==============================] - 2897s 930ms/step - loss: 0.0625 - accuracy: 0.9801 - val_loss: 0.0651 - val_accuracy: 0.9836

Epoch 00002: val_accuracy improved from 0.97761 to 0.98359, saving model to tf2_bert_classifier/weights.h5
Epoch 3/3
3116/3116 [==============================] - 2899s 930ms/step - loss: 0.0604 - accuracy: 0.9812 - val_loss: 0.0717 - val_accuracy: 0.9807

Epoch 00003: val_accuracy did not improve from 0.98359


In [25]:
# 학습과 eval 시작
history = cls_model.fit(train_inputs, train_labels, epochs=5, batch_size=64,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/5
3116/3116 [==============================] - 2898s 930ms/step - loss: 0.0543 - accuracy: 0.9827 - val_loss: 0.0615 - val_accuracy: 0.9831

Epoch 00001: val_accuracy did not improve from 0.98359
Epoch 2/5
3116/3116 [==============================] - 2898s 930ms/step - loss: 0.0523 - accuracy: 0.9834 - val_loss: 0.0707 - val_accuracy: 0.9824

Epoch 00002: val_accuracy did not improve from 0.98359
Epoch 3/5
3116/3116 [==============================] - 2898s 930ms/step - loss: 0.0506 - accuracy: 0.9839 - val_loss: 0.0741 - val_accuracy: 0.9802

Epoch 00003: val_accuracy did not improve from 0.98359
Epoch 4/5
3116/3116 [==============================] - 2897s 930ms/step - loss: 0.0488 - accuracy: 0.9843 - val_loss: 0.0572 - val_accuracy: 0.9855

Epoch 00004: val_accuracy improved from 0.98359 to 0.98554, saving model to tf2_bert_classifier/weights.h5
Epoch 5/5
3116/3116 [==============================] - 2896s 929ms/step - loss: 0.0471 - accuracy: 0.9849 - val_loss: 0.0700 - val_a

In [26]:
# 학습과 eval 시작
history = cls_model.fit(train_inputs, train_labels, epochs=3, batch_size=64,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/3
3116/3116 [==============================] - 2897s 930ms/step - loss: 0.0447 - accuracy: 0.9854 - val_loss: 0.0560 - val_accuracy: 0.9841

Epoch 00001: val_accuracy did not improve from 0.98554
Epoch 2/3
3116/3116 [==============================] - 2896s 929ms/step - loss: 0.0427 - accuracy: 0.9860 - val_loss: 0.0684 - val_accuracy: 0.9823

Epoch 00002: val_accuracy did not improve from 0.98554
Epoch 3/3
3116/3116 [==============================] - 2898s 930ms/step - loss: 0.0418 - accuracy: 0.9861 - val_loss: 0.0574 - val_accuracy: 0.9843

Epoch 00003: val_accuracy did not improve from 0.98554


In [27]:
# 학습과 eval 시작
history = cls_model.fit(train_inputs, train_labels, epochs=3, batch_size=64,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/3
3116/3116 [==============================] - 2899s 930ms/step - loss: 0.0409 - accuracy: 0.9866 - val_loss: 0.0580 - val_accuracy: 0.9818

Epoch 00001: val_accuracy did not improve from 0.98554
Epoch 2/3
3116/3116 [==============================] - 2897s 930ms/step - loss: 0.0406 - accuracy: 0.9865 - val_loss: 0.0473 - val_accuracy: 0.9873

Epoch 00002: val_accuracy improved from 0.98554 to 0.98730, saving model to tf2_bert_classifier/weights.h5
Epoch 3/3
3116/3116 [==============================] - 2898s 930ms/step - loss: 0.0387 - accuracy: 0.9869 - val_loss: 0.0467 - val_accuracy: 0.9861

Epoch 00003: val_accuracy did not improve from 0.98730


In [ ]:
# 학습과 eval 시작
history = cls_model.fit(train_inputs, train_labels, epochs=3, batch_size=64,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/3
3116/3116 [==============================] - 2897s 930ms/step - loss: 0.0368 - accuracy: 0.9878 - val_loss: 0.0473 - val_accuracy: 0.9868

Epoch 00001: val_accuracy did not improve from 0.98730
Epoch 2/3
2695/3116 [========================>.....] - ETA: 6:01 - loss: 0.0358 - accuracy: 0.9880

In [ ]:
cls_model.load_weights('/content/tf2_bert_classifier/new_weights.h5')

In [ ]:
input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

# def clean_text(sent):
#     sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
#     return sent_clean

for test_sent in test['data']:
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAX_LEN=MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
       
    except Exception as e:
        print(e)
        print(test_sent)
        pass
    
test_input_ids=np.array(input_ids, dtype=int)
test_attention_masks=np.array(attention_masks, dtype=int)
test_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
test_inputs=(test_input_ids, test_attention_masks, test_token_type_ids)

In [ ]:
print(train_input_ids[0])
print(train_attention_masks[0])
print(train_token_type_ids[0])
print(tokenizer.decode(train_input_ids[2]))

[   101   9027  21386  29364   9521  16617  17138   9625  16617   9670
  30005   9960 119276   9367  46520  17360   9095  16605   9039 119327
  56645  28188  45725  15001   9570  12310  12424  79604   9367  13764
   9684   9095  16605   9783 119034   9665  12945  29683    102      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
print(tokenizer.decode(21386))

# # 산


In [ ]:
results = cls_model.predict(test_inputs)
results=tf.argmax(results, axis=1)

In [ ]:
sample_submission['label']=results

In [ ]:
sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [ ]:
sample_submission.to_csv('bert_baseline.csv', index=False)

In [ ]:
results = cls_model.predict(train_inputs)
results1=tf.argmax(results, axis=1)

In [ ]:
train['pred_label']=results1

In [ ]:
train[train['pred_label'] != train['label']]

,vocab_eng,vocab_content,label,input_eng,input_all,data,pred_label
202,"['생애 첫 연구사업', '개인기초연구', '이산화탄소', '배출', '저감', '...","['생애 첫 연구사업', '개인기초연구', '이산화탄소', '배출', '저감', '...",0,생애첫연구사업 개인기초연구 이산화탄소 배출 저감 기체 분리 막 시스템 기체 분리 이...,생애첫연구사업 개인기초연구 이산화탄소 배출 저감 기체 분리 막 시스템 기체 분리 이...,생애첫연구사업 개인기초연구 이산화탄소 배출 저감 기체 분리 막 시스템 기체 분리 이...,21
218,"[' 식품위생안전관리', '식품등안전관리', '식품', '잔류농약', '안전', '...","[' 식품위생안전관리', '식품등안전관리', '식품', '잔류농약', '안전', '...",37,식품위생안전관리 식품등안전관리 식품 잔류농약 안전 관리 위해 평가 신규 시험법 확립...,식품위생안전관리 식품등안전관리 식품 잔류농약 안전 관리 위해 평가 신규 시험법 확립...,식품위생안전관리 식품등안전관리 식품 잔류농약 안전 관리 위해 평가 신규 시험법 확립...,26
267,"['원자력원천기술', '원자력기술개발사업', '원자력', '발전소', '작업', '...","['원자력원천기술', '원자력기술개발사업', '원자력', '발전소', '작업', '...",1,원자력원천기술 원자력기술개발사업 원자력 발전소 작업 광학 로봇 개발 원격 제어 힘 ...,원자력원천기술 원자력기술개발사업 원자력 발전소 작업 광학 로봇 개발 원격 제어 힘 ...,원자력원천기술 원자력기술개발사업 원자력 발전소 작업 광학 로봇 개발 원격 제어 힘 ...,0
268,"['시설원예시험연구', '원예특작시험연구RD책임운영', '겨울철', '소과', '종...","['시설원예시험연구', '원예특작시험연구RD책임운영', '겨울철', '소과', '종...",0,시설원예시험연구 원예특작시험연구RD책임운영 겨울철 소과 종 수박 정지 방법 시비 기...,시설원예시험연구 원예특작시험연구RD책임운영 겨울철 소과 종 수박 정지 방법 시비 기...,시설원예시험연구 원예특작시험연구RD책임운영 겨울철 소과 종 수박 정지 방법 시비 기...,24
550,"['기본연구년년', '개인기초연구', '연간', '규모', '사할린', '천연가스'...","['기본연구년년', '개인기초연구', '연간', '규모', '사할린', '천연가스'...",45,기본연구년년 개인기초연구 연간 규모 사할린 천연가스 공급 파이프라인 부식 방지 에너...,기본연구년년 개인기초연구 연간 규모 사할린 천연가스 공급 파이프라인 부식 방지 에너...,기본연구년년 개인기초연구 연간 규모 사할린 천연가스 공급 파이프라인 부식 방지 에너...,13
...,...,...,...,...,...,...,...
174285,"['구제역백신국산화기반기술개발', '농림축산검역검사기술개발', '구제', '역', ...","['구제역백신국산화기반기술개발', '농림축산검역검사기술개발', '구제', '역', ...",0,구제역백신국산화기반기술개발 농림축산검역검사기술개발 구제 역 백신 항원 회수율 저하 ...,구제역백신국산화기반기술개발 농림축산검역검사기술개발 구제 역 백신 항원 회수율 저하 ...,구제역백신국산화기반기술개발 농림축산검역검사기술개발 구제 역 백신 항원 회수율 저하 ...,25
174290,"['년도 창업성장기술개발사업 디딤돌 창업과제 제차 시행계획 수정 공고', '창업성장...","['년도 창업성장기술개발사업 디딤돌 창업과제 제차 시행계획 수정 공고', '창업성장...",19,년도창업성장기술개발사업디딤돌창업과제제차시행계획수정공고 창업성장기술개발 고속 고 내구...,년도창업성장기술개발사업디딤돌창업과제제차시행계획수정공고 창업성장기술개발 고속 고 내구...,년도창업성장기술개발사업디딤돌창업과제제차시행계획수정공고 창업성장기술개발 고속 고 내구...,0
174293,"['년 제차 제품공정개선 기술개발사업', '공정품질기술개발', '현장', '진단',...","['년 제차 제품공정개선 기술개발사업', '공정품질기술개발', '현장', '진단',...",19,년제차제품공정개선기술개발사업 공정품질기술개발 현장 진단 진단 키트 개발 현장 진단 ...,년제차제품공정개선기술개발사업 공정품질기술개발 현장 진단 진단 키트 개발 현장 진단 ...,년제차제품공정개선기술개발사업 공정품질기술개발 현장 진단 진단 키트 개발 현장 진단 ...,0
174294,"['연구시설이용기반구축', '원자력연구기반확충사업', '양성자', '빔', '조사'...","['연구시설이용기반구축', '원자력연구기반확충사업', '양성자', '빔', '조사'...",5,연구시설이용기반구축 원자력연구기반확충사업 양성자 빔 조사 화합물 반도체 테라 헤르츠...,연구시설이용기반구축 원자력연구기반확충사업 양성자 빔 조사 화합물 반도체 테라 헤르츠...,연구시설이용기반구축 원자력연구기반확충사업 양성자 빔 조사 화합물 반도체 테라 헤르츠...,0
